In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import os
ROOT_DIR = os.path.dirname(os.path.abspath(''))
import sys
sys.path.append(ROOT_DIR)

from src.dataset import CustomDataset
from src.fcm import FCM
from src.fcm import train_epoch, eval_epoch, acc, f1
import torch

import matplotlib.pyplot as plt
from PIL import Image
import requests
import torch
from src.dataset import CustomDataset
from torch.utils.data import DataLoader
from torchvision import transforms
import pandas as pd

In [ ]:
DATASET_PATH = os.path.join(ROOT_DIR, 'data', 'MMHS150K', 'MMHS150K_with_img_text.csv')
df = pd.read_csv(DATASET_PATH)

RESULTS_PATH = os.path.join(ROOT_DIR, 'results', 'fcm_predictions.json')

# The JSON is a dictionary with the following structure:
# {
#    "index1": pred1,
#    "index2": pred2,
#    ...
# }
# Load the JSON
import json
with open(RESULTS_PATH, 'r') as f:
    results = json.load(f)
    
# For each index, get the binary_hate and label
keys_results = [int(k) for k in results.keys()]

df_test = df[df["index"].isin(keys_results)]
df_test["pred"] = [results[str(k)] for k in df_test["index"]]

display(df_test.head())

In [ ]:
# Compare "binary_hate" and "pred" to get the accuracy and F1 score
acc_score = acc(df_test["binary_hate"], df_test["pred"])
f1_score = f1(df_test["binary_hate"], df_test["pred"])

print(f"Accuracy: {acc_score}")
print(f"F1 score: {f1_score}")